In [ ]:
from bs4 import BeautifulSoup as bs
import urllib.request
from django.core.validators import URLValidator
from django.core.exceptions import ValidationError
import time
from datetime import datetime

def extractWebsites(url, exceptionURL):
    try:
        webUrl = urllib.request.urlopen(url)
        data = webUrl.read()
        soup = bs(data)
        arr = []
        for link in soup.find_all('a'):
            validate = URLValidator()
            href = link.get('href')
            if (href != None):
                try:
                    validate(href)
                    if (href.find(exceptionURL) == -1):
                        arr.append(href)
                except ValidationError as exception:
                    continue
        return arr
    except:
        return []

def search_array(url, arrOfPoss):
    for index in arrOfPoss:
        if (url.find(index.capitalize())!=-1 or url.find(index.lower())!=-1):
            return True
    return False

data = extractWebsites('https://blog.feedspot.com/usa_news_websites/', 'feedspot.com')
headlines = []

data = data[31:]

other_count = 1
for index in data:
    if (index.find('npr')!=-1 or index.find('NPR')!=-1):
        continue
   
    print("WE ARE ON WEBSITE " + str(other_count))
    new_data = extractWebsites(index, 'interactive')
    covid_urls = []
    arr_keywords = ['coronavirus', 'COVID', 'covid', 'pandemic', 'epidemic', 'disease', 'SARS', 'sars', 'virus', '19']
    for index1 in new_data:
        #print('checking for covid')
        if (search_array(index1, arr_keywords)):
            covid_urls.append(index1)
    count = 0
    #print(covid_urls)
    for val in covid_urls:
        future = time.time() + 600
        print("This is the new count  " + str(count))
        #print('in second loop')
        newer_data = extractWebsites(val, "interactive")
        new_covid_urls = []
        for index1 in newer_data:
            #print('checking for covid')
            #print(index1 + " THIS IS PART OF newer_data")
            if (search_array(index1, arr_keywords)):
                new_covid_urls.append(index1)
        for value in new_covid_urls:
            if (time.time()) > future:
                break
            newest_data = extractWebsites(value, "interactive")
            newest_covid_urls = []
            for index2 in newest_data:
                #print(index2 + " THIS IS PART OF newest_data")
                if (time.time()) > future:
                    break
                #print('checking for covid')
                if (search_array(index2, arr_keywords)):
                    newest_covid_urls.append(index2)
            #print(str(len(newest_covid_urls)) + " lentgth of newest_covid_urls")
            for urls1 in newest_covid_urls:
                if ((time.time()) > future):
                    break
                #print('storing to headlines')
                try:
                    webUrl = urllib.request.urlopen(urls1)
                    data = webUrl.read()
                    soup = bs(data)
                    try:
                        title = soup.find('title').string
                        #print(title + " OVER HERE")
                        if (title not in headlines) and (search_array(title, arr_keywords)):
                            headlines.append(title)
                            with open('headline.txt', 'a') as file:
                                file.write(title+'\n')
                            print(title)
                    except TypeError as exception:
                        print("Exception occured")
                        continue
                except:
                    continue
        for urls in new_covid_urls:
            if (time.time()) > future:
                break
            #print('storing to headlines')
            try:
                webUrl = urllib.request.urlopen(urls)
                data = webUrl.read()
                soup = bs(data)
                try:
                    title = soup.find('title').string
                    if (title not in headlines) and (search_array(title, arr_keywords)):
                        headlines.append(title)
                        with open('headline.txt', 'a') as file:
                            file.write(title+'\n')
                        print(title)
                except:
                    continue
            except:
                continue
        count += 1
    other_count += 1

file.close()

print(len(headlines))
print("WOOHOO IT'S DONE")

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)
